<a href="https://colab.research.google.com/github/ab0473a/Machine-Learning-/blob/main/AutoGluon_EVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 8.3 MB/s 
     |████████████████████████████████| 274 kB 55.0 MB/s 
     |████████████████████████████████| 149 kB 75.7 MB/s 
     |████████████████████████████████| 210 kB 74.9 MB/s 
     |████████████████████████████████| 48 kB 5.7 MB/s 
     |████████████████████████████████| 61 kB 357 kB/s 
     |████████████████████████████████| 65 kB 5.0 MB/s 
     |████████████████████████████████| 132 kB 63.5 MB/s 
     |████████████████████████████████| 802 kB 46.9 MB/s 
     |████████████████████████████████| 1.0 MB 49.6 MB/s 
     |████████████████████████████████| 1.6 MB 61.0 MB/s 
     |████████████████████████████████| 54.5 MB 157 kB/s 
     |████████████████████████████████| 281 kB 72.8 MB/s 
     |████████████████████████████████| 109 kB 72.9 MB/s 
     |████████████████████████████████| 74 kB 4.3 MB/s 
     |████████████████████████████████| 4

In [5]:
%matplotlib inline
import autogluon.core as ag
from autogluon.vision import ImageDataset, ImagePredictor
import pandas as pd

In [ ]:

root = '/content/drive/MyDrive/Colab Notebooks/Emergency_Vehicle_Images'
evi_data = ImageDataset.from_folder(root)
evi_data.head(300)

In [ ]:
evi_data.show_images()

In [11]:
# you can manually split the dataset or use `random_split`
train, val, test = evi_data.random_split(val_size=0.1, test_size=0.1)
print('train #:', len(train), 'test #:', len(test))

train #: 247 test #: 22


In [6]:
predictor = ImagePredictor()
# since the original dataset does not provide validation split, the `fit` function splits it randomly with 90/10 ratio
predictor.fit(evi_data, hyperparameters={'epochs': 25})  

`time_limit=auto` set to `time_limit=7200`.
Reset labels to [0, 1, 2]
Randomly split train_data into train[258]/validation[29] splits.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != resnet50
root.misc.num_workers 4 != 2
root.misc.seed       42 != 120
root.train.epochs    200 != 25
root.train.early_stop_max_value 1.0 != inf
root.train.batch_size 32 != 16
root.train.early_stop_patience -1 != 10
root.train.early_stop_baseline 0.0 != -inf
}
Saved config to /content/6887fe0d/.trial_0/config.yaml
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth
Model resnet50 created, param count:                                         23514179
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.

In [7]:
fit_result = predictor.fit_summary()
print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Top-1 train acc: 0.898, val acc: 0.966


In [26]:
fire_path = '/content/drive/MyDrive/Colab Notebooks/EVI test images/Ferrara-fire-ladder-HD-57.jpeg'
result = predictor.predict(fire_path)
print(result)

0    1
Name: label, dtype: int64


In [22]:
amb_path = '/content/drive/MyDrive/Colab Notebooks/EVI test images/falck amb.jpeg'
result = predictor.predict(amb_path)
print(result)

0    0
Name: label, dtype: int64


In [23]:
pd_path = '/content/drive/MyDrive/Colab Notebooks/EVI test images/crown vic old .jpeg'
result = predictor.predict(pd_path)
print(result)

0    2
Name: label, dtype: int64


In [27]:
proba = predictor.predict_proba(fire_path)
print(proba)

          0         1         2
0  0.369206  0.609261  0.021533


In [28]:
bulk_result = predictor.predict(test)
print(bulk_result)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:946: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


11     0
19     0
28     0
33     0
45     0
50     0
58     0
70     0
71     0
72     0
75     0
84     0
91     0
110    1
133    1
142    1
151    1
167    1
169    1
180    1
200    2
274    2
Name: label, dtype: int64


In [17]:
test_acc = predictor.evaluate(test)
print('Top-1 test acc: %.3f' % test_acc['top1'])

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:946: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
[Epoch 13] validation: top1=1.000000 top5=1.000000


Top-1 test acc: 1.000
